<a href="https://colab.research.google.com/github/ucfilho/raianars_june_2020/blob/master/ANN_parallel_coordenate_jun_20_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/ucfilho/Raianars_paper_One #clonar do Github
%cd Raianars_paper_One

fatal: destination path 'Raianars_paper_One' already exists and is not an empty directory.
/content/Raianars_paper_One


In [2]:
import Go2Ann
import de_soma_Insight
import de_bounds_matriz
import Function
import Go2MinMax
from AvaliaX import AvaliaX
from GeraIndices import GeraIndices

############################### DADOS DE ENTRADA 
RUNS=10 #NÚMERO DE RODADAS
Fc=0.1 # fator de correcao parametros da ANN
its=1 #10 # quantas vezes vai usar o NPAR antes de alterar de novo
ITE=200 #10 # PONTOS quantas vezes vai alteral NPAR 

mut=0.5
crossp=0.5
Fi=mut;CRi=crossp;# para impressão
############################### Função

dim=10
#fchoice='Rastrigin' #fchoice='Michalewicz'
fchoice='Schwefel'

MIN,MAX=Go2MinMax.Intervalo(dim,fchoice)
Fun=Function.Rastrigin
PAR=len(MAX) #NUM DE PARAMETROS A SER OTIMIZADOS

In [3]:
import time
import matplotlib.pyplot as plt
import numpy as np
from random import randint
from scipy import optimize
import pandas as pd
from sklearn.metrics import confusion_matrix

In [4]:
!git clone https://github.com/ucfilho/ANN_read_matrix #clonar do Github
%cd ANN_read_matrix

fatal: destination path 'ANN_read_matrix' already exists and is not an empty directory.
/content/Raianars_paper_One/ANN_read_matrix


In [5]:
ANN_F=pd.read_csv('ANN_INSIGHT_Valor_F.csv')
ANN_CR=pd.read_csv('ANN_INSIGHT_CR.csv')
syn0_F,syn1_F,X_max_F,X_min_F=Go2Ann.GetMatriz(ANN_F)
syn0_CR,syn1_CR,X_max_CR,X_min_CR=Go2Ann.GetMatriz(ANN_CR)

fields = ['DI RELATIVO','FRAC Its','Fo','CRo','F','CR']
Num=2000
Parallel_Coord=np.zeros((Num,6)) #
Max_DIr=X_max_F[0];Max_Fr=X_max_F[1]
Min_DIr=X_min_F[0];Min_Fr=X_min_F[1]
Max_F=X_max_F[2];Max_CR=X_max_F[3]
Min_F=X_min_F[2];Min_CR=X_min_F[3]

In [6]:
for i in range(Num):
  DIr=  Min_DIr+(Max_DIr-Min_DIr)*np.random.random();
  Frac=Min_Fr+(Max_Fr-Min_Fr)*np.random.random();
  Fo=Min_F+(Max_F-Min_F)*np.random.random();
  CRo=Min_CR+(Max_CR-Min_CR)*np.random.random();
  x_train=np.array([DIr,Frac,Fo,CRo])
  x_train=pd.DataFrame(x_train).T
  x_train=Go2Ann.Normatiza(x_train,X_max_F,X_min_F)
  y_calc_F=Go2Ann.ANN_ycal(syn0_F,syn1_F,x_train)
  y_calc_CR=Go2Ann.ANN_ycal(syn0_CR,syn1_CR,x_train)
  y_cod_F=Go2Ann.Classifica(y_calc_F)
  y_cod_CR=Go2Ann.Classifica(y_calc_CR)
  Parallel_Coord[i,0]=DIr;Parallel_Coord[i,1]=Frac;
  Parallel_Coord[i,2]=Fo;Parallel_Coord[i,3]=CRo;
  Parallel_Coord[i,4]=y_cod_F[0];Parallel_Coord[i,5]=y_cod_CR[0];

Parallel_Coord=pd.DataFrame(Parallel_Coord,columns =fields)

In [7]:
print(Parallel_Coord)

      DI RELATIVO  FRAC Its        Fo       CRo    F   CR
0        0.254096  0.677640  0.213985  0.700329  1.0  1.0
1        0.297953  0.471710  0.328250  0.902413 -1.0 -1.0
2        0.348576  0.386625  0.004861  0.295973  1.0  1.0
3        0.253000  0.960381  0.683287  0.090869  1.0 -1.0
4        1.056640  0.888103  0.771952  0.446956 -1.0 -1.0
...           ...       ...       ...       ...  ...  ...
1995     0.975164  0.663228  0.212549  0.169528 -1.0  1.0
1996     1.329981  0.409396  0.928214  0.280402 -1.0 -1.0
1997     0.503336  0.661996  0.198845  0.403277  1.0 -1.0
1998     1.369607  0.949088  0.715932  0.455934 -1.0 -1.0
1999     0.033435  0.875154  0.132944  0.932212  1.0  1.0

[2000 rows x 6 columns]
